In [1]:
# from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import time
from flask import Flask, render_template, redirect
# from flask_pymongo import PyMongo
import pandas as pd

Scrape Diabetes Dataset

In [2]:
# URL of page to be scraped
# url = "https://www.stateofobesity.org/diabetes/" 
    # above url doesn't always pull, downloaded data to html
url = 'resources/diabetes.html'

tables = pd.read_html(url)
#print(tables)

In [3]:
factDF = tables[0]
factDF["State"] = factDF["State"].str[1:]
factDF["Diabetes Rate 2018"] = factDF["Diabetes Rate 2018"].str[:-1]
factDF = factDF.drop(["Trend 1990 - 2018", "95% Confidence Interval"], axis=1)
factDF = factDF.set_index("Rank")
factDF.head()

,State,Diabetes Rate 2018
Rank,,
1,West Virginia,16.2
2,Alabama,14.5
3,Mississippi,14.3
4,Arkansas,14.1
4,Louisiana,14.1


Scrape Population Dataset and combine with our dataframe

In [16]:
stateURL = "https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population"

stateTables = pd.read_html(stateURL)
#print(stateTables)

In [17]:
stateDF = stateTables[0]
stateDF.head()

,"Rank in the fifty states, 2018",Rank in states & territories,Name,"Population estimate, July 1, 2019[4]","Census population, April 1, 2010[5]",Percent increase from 2010-2019[note 1],"Total seats in the U.S. House of Representatives, 2013–2023","Estimated population per electoral vote, 2018[note 2]","Estimated population per House seat, 2018","Census population per House seat, 2010","Percent of the total U.S. population, 2018[note 3]"
0,NaN,NaN,California,39865590,37254523,6.98%,NaN,719219,746359,702885,12.01%
1,NaN,NaN,Texas,29206997,25145561,15.86%,NaN,755312,797273,698503,8.80%
2,NaN,NaN,Florida,21299325,18801310,13.3%,NaN,734459,788864,696468,6.44%
3,NaN,NaN,New York,19542209,19378102,0.8%,NaN,673869,723786,717707,5.91%
4,NaN,NaN,Pennsylvania,12807060,12702379,0.8%,NaN,640353,711503,705715,3.87%


In [18]:
# drop extra columns
stateDF = stateDF.loc[:,['Name',"Population estimate, July 1, 2019[4]"]]

# rename columns
stateDF=stateDF.rename(columns = {"Name":"State","Population estimate, July 1, 2019[4]":'Population_2019'})
stateDF.head()

,State,Population_2019
0,California,39865590
1,Texas,29206997
2,Florida,21299325
3,New York,19542209
4,Pennsylvania,12807060


In [19]:
# get rid of any territories or "Totals" rows
stateDF = stateDF.iloc[:52]
stateDF.head()

,State,Population_2019
0,California,39865590
1,Texas,29206997
2,Florida,21299325
3,New York,19542209
4,Pennsylvania,12807060


In [20]:
# find datatypes and convert if needed
stateDF.dtypes

stateDF.Population_2019 = stateDF.Population_2019.astype('int')

In [21]:
stateDF.dtypes

State              object
Population_2019     int32
dtype: object

In [22]:
#merge stateDF with factDF
factDF = pd.merge(factDF,stateDF, how = 'left', on = 'State')

In [23]:
factDF.head()

,State,Diabetes Rate 2018,Population_2019_x,Population_2019_y
0,West Virginia,16.2,1805832,1805832
1,Alabama,14.5,4887871,4887871
2,Mississippi,14.3,2986530,2986530
3,Arkansas,14.1,3013825,3013825
4,Louisiana,14.1,4659978,4659978


In [24]:
state_abbr_url = "https://simple.wikipedia.org/wiki/U.S._postal_abbreviations"

tables = pd.read_html(state_abbr_url)
state_abbr_df = tables[0].iloc[2:]
state_abbr_df = state_abbr_df.rename(columns={0: "Abbreviation", 1: "State"})
state_abbr_df.head()

,Abbreviation,State
2,AK,Alaska
3,AL,Alabama
4,AR,Arkansas
5,AZ,Arizona
6,CA,California


In [25]:
factDF = pd.merge(factDF,state_abbr_df, how = 'left', on = 'State')

In [26]:
#factDF = factDF.set_index("State")

In [27]:
factDF.head()

,State,Diabetes Rate 2018,Population_2019_x,Population_2019_y,Abbreviation
0,West Virginia,16.2,1805832,1805832,WV
1,Alabama,14.5,4887871,4887871,AL
2,Mississippi,14.3,2986530,2986530,MS
3,Arkansas,14.1,3013825,3013825,AR
4,Louisiana,14.1,4659978,4659978,LA


In [28]:
#factDF.to_csv("static/diabetes.csv")

In [30]:
#Scrape for Candy data now, to join to the factDF

#url for candy Data 
url_halloween = "https://www.candystore.com/blog/facts-trivia/halloween-candy-map-popular/"
url_christmas = "https://www.candystore.com/blog/holidays/christmas-candy-popular-states/"

In [31]:
#read all tables from website and put them into a dataframe
halloween_table = pd.read_html(url_halloween)
christmas_table = pd.read_html(url_christmas)

In [32]:
#define dataframes
halloween_df = halloween_table[0]
christmas_df = christmas_table[0]

halloween_df.head()
#christmas_df.head()

,State,Top Candy,pounds,2nd place,pounds.1,3rd Place,pounds.2
0,AK,Twix,4995,Blow Pops,4396,Milky Way,3634
1,AL,Candy Corn,62067,Tootsie Pops,48386,Hershey's Mini Bars,39391
2,AR,Jolly Ranchers,241483,Hot Tamales,210104,Butterfinger,71776
3,AZ,Snickers,956333,Hot Tamales,766617,Hershey Kisses,502310
4,CA,Skittles,1620736,M&M's,1283715,Assorted Salt Water Taffy,891953


In [33]:
halloween_df.dtypes

State        object
Top Candy    object
pounds        int64
2nd place    object
pounds.1      int64
3rd Place    object
pounds.2      int64
dtype: object

In [37]:
hjoin_df = halloween_df.rename(columns={"State": "Abbreviation"})

In [38]:
#Join with factDF to get diabetes data next to candy data by state

json_df = pd.merge(factDF, hjoin_df, on='Abbreviation', how='outer')

json_df.head()


,State,Diabetes Rate 2018,Population_2019_x,Population_2019_y,Abbreviation,Top Candy,pounds,2nd place,pounds.1,3rd Place,pounds.2
0,West Virginia,16.2,1805832.0,1805832.0,WV,Blow Pops,44123.0,Milky Way,22139.0,Hershey's Mini Bars,18211.0
1,Alabama,14.5,4887871.0,4887871.0,AL,Candy Corn,62067.0,Tootsie Pops,48386.0,Hershey's Mini Bars,39391.0
2,Mississippi,14.3,2986530.0,2986530.0,MS,3 Musketeers,115314.0,Snickers,99497.0,Butterfinger,52733.0
3,Arkansas,14.1,3013825.0,3013825.0,AR,Jolly Ranchers,241483.0,Hot Tamales,210104.0,Butterfinger,71776.0
4,Louisiana,14.1,4659978.0,4659978.0,LA,Lemonheads,108025.0,Reese's Cups,85166.0,Blow Pops,36927.0


In [45]:
#some cleaning - remove unnecessary columns and rename columns
clean_df = json_df.rename(columns={"Diabetes Rate 2018":"Diabetes_Rate_2018","Population_2019_x": 
            "Population_2019","Abbreviation":"State_2","Top Candy":"1st",
            "pounds":"1st_Pounds","pounds.1":"2nd_Pounds","pounds.2":"3rd_Pounds","2nd place":"2nd","3rd Place":"3rd"})

json_final_df = clean_df.drop(columns=['Population_2019_y'])

json_final_df.head()







,State,Diabetes_Rate_2018,Population_2019,State_2,1st,1st_Pounds,2nd,2nd_Pounds,3rd,3rd_Pounds
0,West Virginia,16.2,1805832.0,WV,Blow Pops,44123.0,Milky Way,22139.0,Hershey's Mini Bars,18211.0
1,Alabama,14.5,4887871.0,AL,Candy Corn,62067.0,Tootsie Pops,48386.0,Hershey's Mini Bars,39391.0
2,Mississippi,14.3,2986530.0,MS,3 Musketeers,115314.0,Snickers,99497.0,Butterfinger,52733.0
3,Arkansas,14.1,3013825.0,AR,Jolly Ranchers,241483.0,Hot Tamales,210104.0,Butterfinger,71776.0
4,Louisiana,14.1,4659978.0,LA,Lemonheads,108025.0,Reese's Cups,85166.0,Blow Pops,36927.0


In [46]:
json_final_df.to_csv("data_scrapes/to_json.csv")